# Importing Libraries

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
from datetime import datetime
%matplotlib inline

import pandas_datareader as pdr


import matplotlib.pyplot as plt
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy.stats import norm

import statsmodels.api as sm
import datetime

import talib
import ta

#from talib import EMA, DEMA

#from backtesting import Backtest, Strategy
#from backtesting.lib import crossover
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
from ibapi.utils import iswrapper
import threading
import time
import pandas as pd
import numpy as np
import ta
from IPython.display import clear_output

In [52]:
start_date = "2020-01-01"
end_date = datetime.datetime.today().date().strftime('%Y-%m-%d')


# Fetching TEST data

In [53]:
companies = ['NEM', 'KR', 'AWK', 'NVDA', 'DG'] 
final = pd.DataFrame()
final = final.fillna(0)

In [54]:
for i in companies:
    
    df_d = pdr.get_data_yahoo(i, start_date, end_date)

    df_d.sort_values(["Date"], axis=0, ascending=True, inplace=True)
    df_d.reset_index(inplace = True)
    
    df = pd.DataFrame()
    df = df.fillna(0)
    df = df_d[['Date','Open','High','Low','Close','Volume']]

           
            
    op = df['Open'].astype(float)
    hi = df['High'].astype(float)
    lo = df['Low'].astype(float)
    cl = df['Close'].astype(float)


    #df['Sym']=item 
    #df['Timeframe'] = i



         #------------DOJI---------------#

    #df['CDLDOJI']            = talib.CDLDOJI(op, hi, lo, cl)


    df['ATR'] = talib.ATR(hi, lo, cl)
    df['Stop_Loss'] = df['Close'] - df['ATR']
    df['Take_Profit'] = df['Close'] + 2 * df['ATR']
    df['Entry_Price'] = (df['High'] + df['Low'])/2   
    df['Asset'] = i
    df['UNITS'] = 10   

    
    '''
    df[i+'_Date'] = df['Date']
    df[i+'_Close'] = df['Close']
    df[i+'TP'] = df['Take_Profit']
    df[i+'SL'] = df['Stop_Loss']

    df[i+'_ATR'] = df['ATR']

    df[i+'_Date'] = df['Date']
    '''
    for j, row in df.iterrows():
        
        if df.loc[j,'Entry_Price'] > df.loc[j,'Close']:
            df.loc[j,'Action'] = 'Hold'
        elif df.loc[j,'Entry_Price']<= df.loc[j,'Close']:
            df.loc[j,'Action'] = 'Buy'

    
    
    
    
    df = df.tail(1)
    print(i)      
    
    #df = df[[i+'Date', i+'Close', i+'ATR']]
    #df=df.drop(['Date','Close','Low','Open','High','Volume','Stop_Loss','Take_Profit'], axis=1)

    final =pd.concat([final, df], axis=0)
    #final=final.drop(['Date','Low','Open','High','Volume'], axis=1)
    #final = final.append(df, ignore_index=True)
    final = final[['Date','Asset','Close','Stop_Loss','Take_Profit','Entry_Price','ATR','UNITS','Action']]
    final.reset_index(inplace = True)
 

NEM
KR


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



AWK


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



NVDA


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



DG


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [55]:
final

,index,Date,Asset,Close,Stop_Loss,Take_Profit,Entry_Price,ATR,UNITS,Action
0,0,2020-06-26,NEM,57.980000,55.683057,62.573884,58.109999,2.296942,10,Hold
1,1,2020-06-26,KR,32.935001,31.962591,34.879821,32.762501,0.972410,10,Buy
2,2,2020-06-26,AWK,124.830002,121.285542,131.918921,123.355000,3.544459,10,Buy
3,3,2020-06-26,NVDA,373.549988,360.625902,399.398160,376.545044,12.924086,10,Hold
4,123,2020-06-26,DG,193.320007,188.694390,202.571241,191.805000,4.625617,10,Buy


# IB Order Pacement

In [44]:
class IBapi(EWrapper, EClient):
        
    def __init__(self):
        EClient.__init__(self, self)

    @iswrapper
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId,
                               whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining,
                   'lastFillPrice', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action,
                   order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId,
                   execution.orderId, execution.shares, execution.lastLiquidity)

In [45]:

def run_loop():
    app.run()


def stx_order(symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    contract.primaryExchange = 'NASDAQ'

    return contract


In [ ]:
vinay0028
taurus16

# Placing order in a loop from the PF table

In [47]:
for i, row in final.iterrows():
    
    df = final
    Price = df.loc[i, 'Close']
    SL = df.loc[i, 'Stop_Loss']
    TP = df.loc[i, 'Take_Profit']
    Asset = df.loc[i, 'Asset']
    
    app = IBapi()
    app.connect('127.0.0.1', 7497, 123)

    app.nextorderId = None

    # Start the socket in a thread
    api_thread = threading.Thread(target=run_loop, daemon=True)
    api_thread.start()

    # Check if the API is connected via orderid
    while True:
        if isinstance(app.nextorderId, int):
            print('connected')
            print()
            break
        else:
            print('waiting for connection')
            time.sleep(1)

    # Create order object
    order = Order()
    order.action = 'BUY'
    order.totalQuantity = 5
    order.orderType = 'MIT'
    order.lmtPrice = '153.40'
    order.orderId = app.nextorderId
    app.nextorderId += 1
    order.transmit = False

    # Create stop loss order object
    stop_loss = Order()
    stop_loss.action = 'SELL'
    stop_loss.totalQuantity = 5
    stop_loss.orderType = 'STP'
    stop_loss.auxPrice = '149.2'  # STOP LOSS PRICE
    stop_loss.orderId = app.nextorderId
    app.nextorderId += 1
    stop_loss.parentId = order.orderId
    order.transmit = False

    # Create take profit order object
    take_profit = Order()
    take_profit.action = 'SELL'
    take_profit.totalQuantity = 5
    take_profit.orderType = 'LMT'
    take_profit.lmtPrice = '155'  # TAKE PROFIT PRICE
    app.nextorderId += 1
    take_profit.parentId = order.orderId
    take_profit.transmit = True

    #test mit
    '''
    Order order = new Order();
    order.Action = action;
    order.OrderType = "MIT";
    order.TotalQuantity = quantity;
    order.AuxPrice = price;
    '''

    # place orders
    app.placeOrder(order.orderId, stx_order(Asset), order)
    app.placeOrder(stop_loss.orderId, stx_order(Asset), stop_loss)
    app.placeOrder(trailing.orderId, stx_order(Asset), trailing)
    app.placeOrder(take_profit.orderId, stx_order(Asset), take_profit)

    time.sleep(10)

    # # cancel order
    # print('\nCancelling order...')
    # app.cancelOrder(order.orderId)

    time.sleep(3)
    app.disconnect()

waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection


KeyboardInterrupt: 